<a href="https://colab.research.google.com/github/aKhalid1476/cnn_exoplanet_detector/blob/main/run_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#setup libraries and environment {"display-mode":"form", "form-width":"25%"}
import os
import sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

with HiddenPrints():

    # Installing Streamlit & pyngrok
    !pip -q install streamlit
    !pip -q install pyngrok
    from pyngrok import ngrok
    import streamlit as st
    !wget "https://raw.githubusercontent.com/NolanChai/model_repo/main/income.csv"

def launch_website():
  try:
    if ngrok.get_tunnels():
      ngrok.kill()
    tunnel = ngrok.connect()

    print("Click this link to try your web app:")
    print(tunnel.public_url)

    !streamlit run --server.port 80 app.py >/dev/null # Connect to the URL through Port 80 (>/dev/null hides outputs)

  except KeyboardInterrupt:
    ngrok.kill()

import matplotlib.pyplot as plt
%matplotlib inline

from urllib.request import urlretrieve
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

!wget -q --show-progress 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Planet%20Hunters/exoTrain.csv'
!wget -q --show-progress 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Planet%20Hunters/exoTest.csv'

import seaborn as sns
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn import  metrics
from sklearn import tree
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from scipy.signal import savgol_filter
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score,ConfusionMatrixDisplay,precision_score,recall_score,f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, normalize

import tensorflow as tf
import keras
from keras.models import Sequential
from keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, Conv2D, MaxPooling2D, BatchNormalization, MaxPooling1D
from keras.losses import categorical_crossentropy
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import plotly.graph_objects as go

df_train = pd.read_csv('exoTrain.csv')
df_train['LABEL'] = df_train['LABEL'] -1
df_test = pd.read_csv('exoTest.csv')
df_test['LABEL'] = df_test['LABEL'] - 1

def plot_graphs(history, best):

  plt.figure(figsize=[10,4])
  # summarize history for accuracy
  plt.subplot(121)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy across training\n best accuracy of %.02f'%best[1])
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')

  # summarize history for loss
  plt.subplot(122)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss across training\n best loss of %.02f'%best[0])
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

def analyze_results(model, X_train, y_train, X_test, y_test):
    """
    Helper function to help interpret and model performance.

    Args:
    model: estimator instance
    X_train: {array-like, sparse matrix} of shape (n_samples, n_features)
    Input values for model training.
    y_train : array-like of shape (n_samples,)
    Target values for model training.
    X_test: {array-like, sparse matrix} of shape (n_samples, n_features)
    Input values for model testing.
    y_test : array-like of shape (n_samples,)
    Target values for model testing.

    Returns:
    None
    """
    print("-------------------------------------------")
    print("Model Results")
    print("")
    print("Training:")
    if type(model) == keras.src.engine.sequential.Sequential:
      train_predictions = model.predict(X_train)
      train_predictions = (train_predictions > 0.5)
      cm = confusion_matrix(y_train, train_predictions)
      labels = [0, 1]
      df_cm = pd.DataFrame(cm,index = labels,columns = labels)
      fig = plt.figure()
      res = sns.heatmap(df_cm, annot=True,cmap='Blues', fmt='g')
      #plt.yticks([1.25, 3.75], labels,va='center')
      plt.title('Confusion Matrix - Test Data')
      plt.ylabel('True label')
      plt.xlabel('Predicted label')
      plt.show()
    else:
      plt.close()
      ConfusionMatrixDisplay.from_estimator(model,X_train,y_train)
      plt.show()

    print("Testing:")
    if type(model) == keras.src.engine.sequential.Sequential:
      test_predictions = model.predict(X_test)
      test_predictions = (test_predictions > 0.5)
      cm = confusion_matrix(y_test, test_predictions)
      labels = [0, 1]
      df_cm = pd.DataFrame(cm,index = labels,columns = labels)
      fig = plt.figure()
      res = sns.heatmap(df_cm, annot=True,cmap='Blues', fmt='g')
      #plt.yticks([1.25, 3.75], labels,va='center')
      plt.title('Confusion Matrix - Test Data')
      plt.ylabel('True label')
      plt.xlabel('Predicted label')
      plt.show()
    else:
      ConfusionMatrixDisplay.from_estimator(model,X_test,y_test)

X_train = df_train.drop('LABEL', axis=1)
y_train = df_train['LABEL']
X_test = df_test.drop('LABEL', axis=1)
y_test = df_test['LABEL']

# Preprocess Data
# Helper functions that we can run for the three augmentation functions that will be used, but not explroed in depth

def smote(a,b):
    model = SMOTE()
    X,y = model.fit_resample(a, b)
    return X,y

def savgol(df1,df2):
    x = savgol_filter(df1,21,4,deriv=0)
    y = savgol_filter(df2,21,4,deriv=0)
    return x,y

def fourier(df1,df2):
    X_train = np.abs(np.fft.fft(df1, axis=1))
    X_test = np.abs(np.fft.fft(df2, axis=1))
    return X_train,X_test

def norm(df1,df2):
    X_train = normalize(df1)
    X_test = normalize(df2)
    return X_train,X_test

def robust(df1,df2):
    scaler = RobustScaler()
    X_train = scaler.fit_transform(df1)
    X_test = scaler.transform(df2)
    return X_train,X_test

fourier_X_train, fourier_X_test = fourier(X_train, X_test)
savgol_X_train, savgol_X_test = savgol(fourier_X_train, fourier_X_test)
norm_X_train, norm_X_test = norm(savgol_X_train,savgol_X_test)
robust_X_train, robust_X_test = robust(norm_X_train, norm_X_test)
smote_X_train,smote_y_train = smote(robust_X_train, y_train)

# Here we're adding the generated, augmented data onto the testing data
# aug_X_train, new_X_test_data, aug_y_train, new_y_test_data = train_test_split(smote_X_train, smote_y_train, test_size=0.3)
# aug_X_test = np.concatenate((robust_X_test, new_X_test_data), axis=0)
# aug_y_test = np.concatenate((y_test, new_y_test_data), axis=0)

def preprocess_data(df):
    """Apply preprocessing steps to the dataframe."""
    X = df.drop('LABEL', axis=1).values
    y = df['LABEL'].values
    # Fourier transform
    X = np.abs(np.fft.fft(X, axis=1))
    # Savitzky-Golay filter
    X = savgol_filter(X, 21, 4, deriv=0, axis=1)
    # Normalize
    X = normalize(X)
    # Robust scaling
    scaler = RobustScaler()
    X = scaler.fit_transform(X)
    # SMOTE
    smote = SMOTE()
    X, y = smote.fit_resample(X, y)
    # Expand dimensions for CNN input
    X_cnn = np.expand_dims(X, axis=2)
    return X, X_cnn, y

exoTrain.csv        100%[===================>] 250.08M   120MB/s    in 2.1s    
exoTest.csv         100%[===================>]  27.57M  59.7MB/s    in 0.5s    


In [ ]:
# Testing
%%writefile app.py
import streamlit as st
st.title('Hello World!')

Writing app.py


In [ ]:
!ngrok authtoken 32FUmgOExxNUtRGr4TRFHza5mqc_6rTFQxmvjn9rq8BLPHmrH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
launch_website()

Click this link to try your web app:
https://4d04aca2bb79.ngrok-free.app


In [ ]:
from google.colab import drive
import zipfile

drive.mount('/content/gdrive')
cnn_path = '/content/gdrive/My Drive/cnn_exoplanets.zip'

with zipfile.ZipFile(cnn_path, 'r') as zip_ref:
    zip_ref.extractall('')

model = tf.keras.models.load_model('cnn/cnn.keras')
drive.flush_and_unmount()

model.summary()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 3193, 8)        │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 799, 8)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 799, 16)        │           400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 200, 16)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         3,201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,949 (42.77 KB)

 Trainable params: 3,649 (14.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,300 (28.52 KB)

In [ ]:
# preprocess data
X_train, X_train_cnn, y_train = preprocess_data(df_train)

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.graph_objects as go
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from scipy.signal import savgol_filter
from sklearn.preprocessing import normalize

# Load data
df_train = pd.read_csv('exoTrain.csv')
df_train['LABEL'] = df_train['LABEL'] - 1

# Load the CNN model
model = tf.keras.models.load_model('cnn/cnn.keras')

def preprocess_data(df):
    """Apply preprocessing steps to the dataframe."""
    X = df.drop('LABEL', axis=1).values
    y = df['LABEL'].values
    # Fourier transform
    X = np.abs(np.fft.fft(X, axis=1))
    # Savitzky-Golay filter
    X = savgol_filter(X, 21, 4, deriv=0, axis=1)
    # Normalize
    X = normalize(X)
    # Robust scaling
    scaler = RobustScaler()
    X = scaler.fit_transform(X)
    # SMOTE
    smote = SMOTE()
    X, y = smote.fit_resample(X, y)
    # Expand dimensions for CNN input
    X_cnn = np.expand_dims(X, axis=2)
    return X, X_cnn, y

X_train, X_train_cnn, y_train = preprocess_data(df_train)

def predict(index):
    """Make a prediction using the Keras model."""
    tensor = X_train[index].reshape(1, -1, 1)
    output = model.predict(tensor)
    return output.flatten()[0]

################################################################################
##########=                   (Light Curves)                   =################
################################################################################
st.title('Exoplanet Light Curve Visualization with CNN Predictions')

# Slider for selecting the index of the light curve
index = st.slider("Select Index for Light Curve", min_value=0, max_value=len(X_train)-1, value=12, step=1)

# Display CNN prediction results
prediction = predict(index)
st.write(f"Prediction (probability of being an exoplanet): {prediction:.4f}")
t_0 = st.slider("Start of Period (t_0)", min_value=0, max_value=3197, value=430, step=1)
period = st.slider("Length of Period", min_value=0, max_value=3197, value=1184, step=1)

# Create Plotly graph for the full light curve with a rectangle highlighting the period
fig = go.Figure()
fig.add_trace(go.Scatter(y=X_train[index].flatten(), mode='lines', name='Light Curve'))
fig.add_shape(type="rect",
              x0=t_0,
              y0=min(X_train[index])-5,
              x1=t_0+period,
              y1=max(X_train[index])+5,
              line=dict(color="Red"),
              fillcolor="LightPink",
              opacity=0.5)
fig.update_layout(title="Box Covering One Period of Exoplanet Transit",
                  xaxis_title="Observation Point",
                  yaxis_title="Normalized Flux",
                  showlegend=True)
st.plotly_chart(fig, use_container_width=True)

# Create Plotly graph for just the selected period
fig_period = go.Figure()
fig_period.add_trace(go.Scatter(y=X_train[index, t_0: t_0+period].flatten(), mode='lines', name='Selected Period'))
fig_period.update_layout(title="Plot of Just One Period",
                         xaxis_title="Observation Point",
                         yaxis_title="Normalized Flux",
                         showlegend=True)
st.plotly_chart(fig_period, use_container_width=True)


Overwriting app.py


In [ ]:
launch_website()

Click this link to try your web app:
https://108bccd31ac8.ngrok-free.app
2025-09-04 20:11:02.004647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757016662.048998    4140 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757016662.061971    4140 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757016662.098649    4140 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757016662.098733    4140 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17

2025-09-04 20:11:13.301335: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import tensorflow as tf
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from scipy.signal import savgol_filter
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Load data
df_train = pd.read_csv('exoTrain.csv')
df_train['LABEL'] = df_train['LABEL'] - 1

# Load the CNN model
cnn_model = tf.keras.models.load_model('cnn/cnn.keras')

def preprocess_data(df):
    """Apply preprocessing steps to the dataframe."""
    X = df.drop('LABEL', axis=1).values
    y = df['LABEL'].values
    # Fourier transform
    X = np.abs(np.fft.fft(X, axis=1))
    # Savitzky-Golay filter
    X = savgol_filter(X, 21, 4, deriv=0, axis=1)
    # Normalize
    X = normalize(X)
    # Robust scaling
    scaler = RobustScaler()
    X = scaler.fit_transform(X)
    # SMOTE
    smote = SMOTE()
    X, y = smote.fit_resample(X, y)
    # Expand dimensions for CNN input
    X_cnn = np.expand_dims(X, axis=2)
    return X, X_cnn, y

X_train, X_train_cnn, y_train = preprocess_data(df_train)

def predict_cnn(index):
    """Make a prediction using the CNN model."""
    tensor = X_train_cnn[index].reshape(1, -1, 1)
    output = cnn_model.predict(tensor)
    return output.flatten()[0]

st.title('Exoplanet Light Curve Visualization with CNN Predictions')

# Slider for selecting the index of the light curve
index = st.slider("Select Index for Light Curve", min_value=0, max_value=len(X_train)-1, value=12, step=1)

# Display CNN prediction results
cnn_prediction = predict_cnn(index)
st.write(f"CNN Prediction (probability of being an exoplanet): {cnn_prediction:.4f}")

t_0 = st.slider("Start of Period (t_0)", min_value=0, max_value=3197, value=430, step=1)
period = st.slider("Length of Period", min_value=0, max_value=3197, value=1184, step=1)

# Checkbox widgets for graph selection
show_full_light_curve = st.checkbox("Show Full Light Curve", value=True)
show_selected_period = st.checkbox("Show Selected Period", value=True)
show_confusion_matrix = st.checkbox("Show Confusion Matrix", value=True)

# Plotly graph for full light curve with rectangle highlighting the period
if show_full_light_curve:
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=X_train[index].flatten(), mode='lines', name='Light Curve'))
    fig.add_shape(type="rect",
                  x0=t_0,
                  y0=min(X_train[index])-5,
                  x1=t_0+period,
                  y1=max(X_train[index])+5,
                  line=dict(color="Red"),
                  fillcolor="LightPink",
                  opacity=0.5)
    fig.update_layout(title="Box Covering One Period of Exoplanet Transit",
                      xaxis_title="Observation Point",
                      yaxis_title="Normalized Flux",
                      showlegend=True)
    st.plotly_chart(fig, use_container_width=True)

# Plotly graph for just the selected period
if show_selected_period:
    fig_period = go.Figure()
    fig_period.add_trace(go.Scatter(y=X_train[index, t_0: t_0+period], mode='lines', name='Selected Period'))
    fig_period.update_layout(title="Plot of Just One Period",
                             xaxis_title="Observation Point",
                             yaxis_title="Normalized Flux",
                             showlegend=True)
    st.plotly_chart(fig_period, use_container_width=True)

# Compute and plot confusion matrices
cnn_pred_labels = (cnn_model.predict(X_train_cnn).flatten() > 0.5).astype(int)
cm_cnn = confusion_matrix(y_train, cnn_pred_labels)

# Confusion Matrix for CNN
if show_confusion_matrix:
    fig_cm_cnn = ff.create_annotated_heatmap(z=cm_cnn, x=['Pred 0', 'Pred 1'], y=['True 0', 'True 1'],
                                             colorscale='Viridis', showscale=True)
    fig_cm_cnn.update_layout(title='Confusion Matrix for CNN for Entire Dataset')
    st.plotly_chart(fig_cm_cnn, use_container_width=True)


Overwriting app.py


In [ ]:
launch_website()

Click this link to try your web app:
https://1421d2073687.ngrok-free.app
2025-09-04 20:13:05.123682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757016785.172226    4669 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757016785.186406    4669 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757016785.231741    4669 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757016785.231807    4669 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:17